In [ ]:
import numpy as np
import seaborn
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
import imageio
import glob
from IPython import display
from matplotlib import cm
import time

seaborn.set()
np.set_printoptions(suppress=True)

import utils
import clustering_functions

In [ ]:
dataset1, dataset2, dataset3 = utils.read_dataset2()
dataset1, dataset2, dataset3 = utils.normalize_dataset2(dataset1, dataset2, dataset3)

color_list = ['firebrick','darkorange','darkgoldenrod','forestgreen','dodgerblue','blueviolet','magenta', 'black']
utils.plot_dataset2(dataset1, dataset2, dataset3, color_list)

In [ ]:
clustering_functions.kmeans(dataset1, k=7, dataname="dataset1", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/kmeans_dataset1.gif')

In [ ]:
clustering_functions.kmeans(dataset2, k=3, dataname="dataset2", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/kmeans_dataset2.gif')

In [ ]:
clustering_functions.kmeans(dataset3, k=2, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/kmeans_dataset3.gif')

In [ ]:
%timeit clustering_functions.kmeans(dataset1, k=7, dataname="dataset1", create_anim_file=False, print_output=False)

In [ ]:
%timeit clustering_functions.kmeans(dataset2, k=3, dataname="dataset2", create_anim_file=False, print_output=False)

In [ ]:
%timeit clustering_functions.kmeans(dataset3, k=2, dataname="dataset3", create_anim_file=False, print_output=False)

In [ ]:
clustering_functions.single_linkage(dataset1, nb_of_clusters=7, dataname="dataset1", create_anim_file=True, plot_every_iter=50, color_list=color_list)
display.Image(filename='gif/singlelinkage_dataset1.gif')

In [ ]:
clustering_functions.single_linkage(dataset2, nb_of_clusters=3, dataname="dataset2", create_anim_file=True, plot_every_iter=20, color_list=color_list)
display.Image(filename='gif/singlelinkage_dataset2.gif')

In [ ]:
clustering_functions.single_linkage(dataset3, nb_of_clusters=2, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/singlelinkage_dataset3.gif')

In [ ]:
clustering_functions.complete_linkage(dataset1, nb_of_clusters=7, dataname="dataset1", create_anim_file=True, plot_every_iter=50, color_list=color_list)
display.Image(filename='gif/completelinkage_dataset1.gif')

In [ ]:
clustering_functions.complete_linkage(dataset2, nb_of_clusters=3, dataname="dataset2", create_anim_file=True, plot_every_iter=20, color_list=color_list)
display.Image(filename='gif/completelinkage_dataset2.gif')

In [ ]:
clustering_functions.complete_linkage(dataset3, nb_of_clusters=2, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/completelinkage_dataset3.gif')

In [ ]:
clustering_functions.group_average(dataset1, nb_of_clusters=7, dataname="dataset1", create_anim_file=True, plot_every_iter=50, color_list=color_list)
display.Image(filename='gif/groupaverage_dataset1.gif')

In [ ]:
clustering_functions.group_average(dataset2, nb_of_clusters=3, dataname="dataset2", create_anim_file=True, plot_every_iter=20, color_list=color_list)
display.Image(filename='gif/groupaverage_dataset2.gif')

In [ ]:
clustering_functions.group_average(dataset3, nb_of_clusters=2, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/groupaverage_dataset3.gif')

In [ ]:
def dbscan(dataset, min_pts, eps, create_anim_file=False, dataname="data"):
    start_time = time.time()
    eps_c = eps * eps

    core_point_list = []
    for point_index, cur_point in enumerate(dataset):
        points_within_eps = 0
        for other_point_index, other_point in enumerate(dataset):
            if point_index == other_point_index:
                continue
            if np.square(cur_point - other_point).sum() < eps_c:
                points_within_eps += 1
            if points_within_eps >= min_pts:
                core_point_list.append(point_index)
                break

    core_points = dataset[core_point_list]
    non_core_points = np.delete(dataset, core_point_list, axis=0)

    border_point_list = []
    for non_core_point_index, non_core_point in enumerate(non_core_points):
        for core_point in core_points:
            if np.square(non_core_point - core_point).sum() < eps_c:
                border_point_list.append(non_core_point_index)
                break

    border_points = non_core_points[border_point_list]
    noise_points = np.delete(non_core_points, border_point_list, axis=0)

    cur_cluster = -1
    clusters = np.full(len(dataset), cur_cluster)
    unique_clusters = np.unique(clusters)
    
    done = False
    i = 0
    while not done:
         
        if create_anim_file and len(unique_clusters) <= 8:
            color_arr = [color_list[cluster] for cluster in clusters]
            plt.scatter(dataset[:,0], dataset[:,1], c=color_arr)
            plt.title(f'{dataname}')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.savefig(f'gif/dbscan/{dataname}/{i:04d}.png')
            plt.close()
        
        
        occurence = np.nonzero(clusters[core_point_list] == -1)
        if len(occurence[0]) > 0:
            core_point_occurence = occurence[0][0]
            cur_cluster += 1
            clusters[core_point_list[core_point_occurence]] = cur_cluster

            core_point = dataset[core_point_list[core_point_occurence]]
            other_core_indices = []
            for other_point_index, other_point in enumerate(dataset):
                if clusters[other_point_index] == -1 and np.square(core_point - other_point).sum() < eps_c:
                    clusters[other_point_index] = clusters[core_point_list[core_point_occurence]]
                    if other_point_index in core_point_list:
                        other_core_indices.append(other_point_index)
            for other_core_index in other_core_indices:
                new_core_point = dataset[other_core_index]
                for other_point_index, other_point in enumerate(dataset):
                    if clusters[other_point_index] == -1 and np.square(new_core_point - other_point).sum() < eps_c:
                        clusters[other_point_index] = clusters[other_core_index]
                        if other_point_index in core_point_list:
                            other_core_indices.append(other_point_index)
        else:
            done = True
        
        i += 1
            
    unique_clusters = np.unique(clusters)
    squared_errors = 0
    for i in unique_clusters:
        if i == -1:
            continue
        cluster_centroid = dataset[clusters == i].mean(axis=0)
        squared_error = np.square(dataset[clusters == i] - cluster_centroid).sum()
        squared_errors += squared_error
    
    end_time = time.time()
    print(f"Sum of squared errors for {dataname} (normalized) with DBSCAN:{squared_errors:.4f}")
    print(f"DBSCAN for {dataname} took :{end_time - start_time:.3f} seconds")

In [ ]:
create_anim_file = True

In [ ]:
dbscan(dataset1, min_pts=230, eps=0.795, create_anim_file=create_anim_file, dataname="dataset1")

In [ ]:
dbscan(dataset2, min_pts=30, eps=0.4, create_anim_file=create_anim_file, dataname="dataset2")

In [ ]:
dbscan(dataset3, min_pts=45, eps=0.927, create_anim_file=create_anim_file, dataname="dataset3")

In [ ]:
if create_anim_file:
    anim_file = 'gif/dbscan_dataset1.gif'

    frames = []
    filenames = glob.glob('gif/dbscan/dataset1/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(3):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=2)
    
anim_file = 'gif/dbscan_dataset1.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/dbscan_dataset2.gif'

    frames = []
    filenames = glob.glob('gif/dbscan/dataset2/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(3):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=2)
    
anim_file = 'gif/dbscan_dataset2.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/dbscan_dataset3.gif'

    frames = []
    filenames = glob.glob('gif/dbscan/dataset3/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(3):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=2)
    
anim_file = 'gif/dbscan_dataset3.gif'
display.Image(filename=anim_file)